In [1]:
import mxnet as mx
from mxnet import ndarray as nd
from mxnet import autograd as ag
from mxnet import init
from mxnet import gluon
from mxnet.gluon.data import vision
from mxnet.gluon.data.vision import transforms
from mxnet.gluon.data import DataLoader
from mxnet.gluon import nn
import datetime
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
train_augs = transforms.Compose([
    transforms.Resize(224), 
    transforms.RandomResizedCrop(224), 
    transforms.RandomFlipLeftRight(), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

valid_augs = transforms.Compose([
    transforms.Resize(224), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

In [3]:
data_dir = 'data/cifar10/'
batch_size = 32

train_ds = vision.ImageFolderDataset(root=data_dir+'train', flag=1)
valid_ds = vision.ImageFolderDataset(root=data_dir+'valid', flag=1)

train_data = DataLoader(train_ds.transform_first(train_augs), 
                        batch_size=batch_size, shuffle=True, last_batch='keep')
valid_data = DataLoader(valid_ds.transform_first(valid_augs), 
                        batch_size=batch_size, shuffle=False, last_batch='keep')

In [4]:
print(len(train_ds))
print(len(train_data))
for data, label in train_data:
    print(data.shape)
    print(label.shape)
    print(label.astype('float32').as_in_context(mx.gpu(0)))
    break

45000
1407
(32, 3, 224, 224)
(32,)

[ 5.  6.  2.  1.  1.  0.  9.  6.  5.  1.  9.  3.  1.  5.  3.  4.  2.  6.
  3.  4.  3.  8.  2.  6.  5.  0.  4.  7.  9.  8.  8.  3.]
<NDArray 32 @gpu(0)>


## 设计模型 --Vgg-16

In [5]:
class Vgg16(nn.HybridBlock):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(Vgg16, self).__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            net = nn.HybridSequential()
            for _ in range(2):
                net.add(nn.Conv2D(channels=64, kernel_size=3, 
                                  strides=1, padding=1))
                net.add(nn.BatchNorm())
                net.add(nn.Activation(activation='relu'))
            net.add(nn.MaxPool2D(pool_size=2, strides=2, padding=0))
            for _ in range(2):
                net.add(nn.Conv2D(channels=128, kernel_size=3, 
                                  strides=1, padding=1))
                net.add(nn.BatchNorm())
                net.add(nn.Activation(activation='relu'))
            net.add(nn.MaxPool2D(pool_size=2, strides=2, padding=0))
            for _ in range(3):
                net.add(nn.Conv2D(channels=256, kernel_size=3, 
                                  strides=1, padding=1))
                net.add(nn.BatchNorm())
                net.add(nn.Activation(activation='relu'))
            net.add(nn.MaxPool2D(pool_size=2, strides=2, padding=0))
            for _ in range(3):
                net.add(nn.Conv2D(channels=512, kernel_size=3, 
                                  strides=1, padding=1))
                net.add(nn.BatchNorm())
                net.add(nn.Activation(activation='relu'))
            net.add(nn.MaxPool2D(pool_size=2, strides=2, padding=0))
            for _ in range(3):
                net.add(nn.Conv2D(channels=512, kernel_size=3, 
                                  strides=1, padding=1))
                net.add(nn.BatchNorm())
                net.add(nn.Activation(activation='relu'))
            net.add(nn.MaxPool2D(pool_size=2, strides=2, padding=0))
            net.add(nn.Flatten())
            net.add(nn.Dense(4096))
            net.add(nn.BatchNorm())
            net.add(nn.Activation(activation='relu'))
            net.add(nn.Dropout(0.5))
            net.add(nn.Dense(4096))
            net.add(nn.BatchNorm())
            net.add(nn.Activation(activation='relu'))
            net.add(nn.Dropout(0.5))
            net.add(nn.Dense(num_classes))
        self.net = net
        
    def hybrid_forward(self, F, x):
        for i, layer in enumerate(self.net):
            out = layer(x)
            x = out
            if self.verbose:
                print('Block %d, Output: %s' % (i+1, out.shape))
        return out

In [6]:
def get_net(ctx, num_classes):
    net = Vgg16(num_classes=num_classes)
    net.initialize(ctx=ctx, init=init.Xavier())
    return net

In [7]:
# net = Vgg16(100, verbose=True)
# net.initialize()
# net

In [ ]:
# for data, label in train_data:
#     output = net(data)
#     break

In [ ]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
ctx = mx.gpu(0)
net = get_net(ctx, num_classes=10)
net.hybridize()

utils.train(net, train_data, valid_data, ctx=ctx, num_epoches=100, softmax_cross_entropy=softmax_cross_entropy,
            optimizer='momentum', lr=0.01, lr_decay=0.1, lr_period=50, momentum=0.9, weight_decay=5e-4, 
            cost_peroid=10, print_cost=True)

Epoch 1, Train_loss: 4.53740164174, Train_acc: 0.154761904762, Valid_acc: 0.189888535032, Time 00:12:29, lr: 0.010000
Epoch 2, Train_loss: 2.56480705933, Train_acc: 0.205246090974, Valid_acc: 0.323248407643, Time 00:07:24, lr: 0.010000
Epoch 3, Train_loss: 2.09467909542, Train_acc: 0.286225124378, Valid_acc: 0.383359872611, Time 00:07:24, lr: 0.010000
Epoch 4, Train_loss: 1.85151287626, Train_acc: 0.322272565743, Valid_acc: 0.43053343949, Time 00:07:25, lr: 0.010000
Epoch 5, Train_loss: 1.89671210647, Train_acc: 0.35743159204, Valid_acc: 0.479299363057, Time 00:07:24, lr: 0.010000
Epoch 6, Train_loss: 1.67809216748, Train_acc: 0.395700071073, Valid_acc: 0.504777070064, Time 00:07:25, lr: 0.010000
Epoch 7, Train_loss: 1.65086909653, Train_acc: 0.439543354655, Valid_acc: 0.551154458599, Time 00:07:25, lr: 0.010000
Epoch 8, Train_loss: 1.53989702041, Train_acc: 0.473214285714, Valid_acc: 0.610071656051, Time 00:07:25, lr: 0.010000
Epoch 9, Train_loss: 1.23853374236, Train_acc: 0.507173951

Epoch 70, Train_loss: 0.340032949322, Train_acc: 0.875799573561, Valid_acc: 0.931727707006, Time 00:07:24, lr: 0.001000
Epoch 71, Train_loss: 0.359574322811, Train_acc: 0.877887348969, Valid_acc: 0.928542993631, Time 00:07:24, lr: 0.001000
Epoch 72, Train_loss: 0.328870334511, Train_acc: 0.876465884861, Valid_acc: 0.930334394904, Time 00:07:24, lr: 0.001000
Epoch 73, Train_loss: 0.379516922244, Train_acc: 0.8786202914, Valid_acc: 0.931329617834, Time 00:07:24, lr: 0.001000


KeyboardInterrupt: 

In [ ]:
utils.train(net, train_data, valid_data, ctx=ctx, num_epoches=20, softmax_cross_entropy=softmax_cross_entropy,
            optimizer='momentum', lr=0.0001, lr_decay=0.01, lr_period=50, momentum=0.9, weight_decay=5e-4, 
            cost_peroid=10, print_cost=True)